# 준비사항
1. google earthengine 계정 및 프로젝트 GCP 콘솔 연결 (실험 시 구글계정으로 가능)
2. Mapbox API token (가시화 경우에 사용, 파일저장 시 X)

## 입력
1. alpha value (유의지수, 일반적으로 0.0001 사용, 과소추정 시 변경)
2. 분석 대상지역 geojson (다각형도 무관함, wgs-84 degree 사용할 것)
3. 통계분석 geojson
    - 주의사항 : 분석 대상지역에서 육지와 호소가 포함된 지역으로 1km x 1km 내외로 선택
4. relative orbit number (Sentinel-1 커버리지, 대상지역의 주된 path 번호를 확인)
    - 주의사항 : ASC, DESC, A/B 모두 다름 사전에 정의되어야 함
5. Direction (ASCENDING, DESCENDING 선택)
    - 주의사항 : ASC, DESC 중 택하여 분석을 진행. 단, 발생시기 추정 시 ASC, DESC 모두 사용
6. DateTime (시작 및 종료기간)
    - 주의사항 : 영상 검색 시 중복된 DateTime이 있다면 분석 대상지역 geojson 조정해야 함


## References
- Moreira, Alberto and Prats-Iraola, Pau and Younis, Marwan and Krieger, Gerhard and Hajnsek, Irena and Papathanassiou, Konstantinos (2013) A Tutorial on Synthetic Aperture Radar. IEEE Geoscience and Remote Sensing Magazine (GRSM), 1 (1), pp. 6-43. IEEE - Institute of Electrical and Electronics Engineers. doi: 10.1109/MGRS.2013.2248301. ISSN 2168-6831.
- K. Conradsen; A.A. Nielsen; J. Schou; H. Skriver (2003) A test statistic in the complex Wishart distribution and its application to change detection in polarimetric SAR data. IEEE Transactions on Geoscience and Remote Sensing (GRSM), 41(1), pp. 4-19. IEEE - Institute of Electrical and Electronics Engineers. doi: 10.1109/TGRS.2002.808066.
- Morton J. Canty 1, Allan A. Nielsen, Knut Conradsen and Henning Skriver (2019) Statistical Analysis of Changes in Sentinel-1 Time Series on the Google Earth Engine. Remote Sensing, 12(1), MDPI. doi: 10.3390/rs12010046.
- Allan A. Nielsen, Henning Skriver, Knut Conradsen (2020) The Loewner Order and Direction of Detected Change in Sentinel-1 and Radarsat-2 Data. IEEE Geoscience and Remote Sensing Letters, 17(2), pp. 242-246. IEEE - Institute of Electrical and Electronics Engineers. doi: 10.1109/LGRS.2019.2918636.
- Allan A. Nielsen, Knut Conradsen, Henning Skriver (2016) Omnibus test for change detection in a time sequence of polarimetric SAR data, IEEE International Symposium on Geoscience and Remote Sensing (IGARSS),  IEEE - Institute of Electrical and Electronics Engineers. doi: 10.1109/IGARSS.2016.7729878



# package, api 설정

In [ ]:
import ee
import numpy as np
import folium
import matplotlib.pyplot as plt
import branca.colormap as cm
from scipy.stats import chi2
from datetime import datetime, timedelta


%matplotlib inline

# service_account = 'GCP 어카운트 주소'
# credentials = ee.ServiceAccountCredentials(service_account, '어카운트 키.json')
# ee.Initialize(credentials)


token = "pk.." # your mapbox token
# https://docs.mapbox.com/api/maps/styles/
tileurl = 'https://api.mapbox.com/styles/v1/mapbox/dark-v10/tiles/{z}/{x}/{y}?access_token=' + str(token)

ee.Authenticate()
ee.Initialize()

# 함수호출

In [ ]:
def xz_abc_layer(self, xyz_image_obj, vis_params, layer_name):
    map_id_data = ee.Image(xyz_image_obj).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_data['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=layer_name,
        overlay=True,
        control=True).add_to(self)

folium.Map.xz_abc_layer = xz_abc_layer

def qwe2cdf(qwe2, dof):
    return ee.Image(qwe2.divide(2)).gammainc(ee.Number(dof).divide(2))

def yz(im):
    return im.expression('b(0)*b(1)')

def clip_image(img):
    return ee.Image(img).clip(aoi)

def select_aa(current):
    return ee.Image(current).select('VV')

def abc_log_sum(im_list, j):
    im_list = ee.List(im_list)
    sum_j = ee.ImageCollection(im_list.slice(0, j)).reduce(ee.Reducer.sum())
    return ee.Image(yz(sum_j)).log()

def abc_log(im_list, j):
    im = ee.Image(ee.List(im_list).get(j.subtract(1)))
    return ee.Image(yz(im)).log()

def p_value(im_list, j, m=4.4):
    im_list = ee.List(im_list)
    j = ee.Number(j)
    m2logRj = (abc_log_sum(im_list, j.subtract(1))
               .multiply(j.subtract(1))
               .add(abc_log(im_list, j))
               .add(ee.Number(2).multiply(j).multiply(j.log()))
               .subtract(ee.Number(2).multiply(j.subtract(1))
               .multiply(j.subtract(1).log()))
               .subtract(abc_log_sum(im_list, j).multiply(j))
               .multiply(-2).multiply(m))
    pv = ee.Image.constant(1).subtract(qwe2cdf(m2logRj, 2))
    return (pv, m2logRj)

def calc_p_value(im_list, j, m=4.4):
    im_list = ee.List(im_list)
    j = ee.Number(j)
    m2logRj = (abc_log_sum(im_list, j.subtract(1))
               .multiply(j.subtract(1))
               .add(abc_log(im_list, j))
               .add(ee.Number(2).multiply(j).multiply(j.log()))
               .subtract(ee.Number(2).multiply(j.subtract(1))
               .multiply(j.subtract(1).log()))
               .subtract(abc_log_sum(im_list, j).multiply(j))
               .multiply(-2).multiply(m))
    pv = ee.Image.constant(1).subtract(qwe2cdf(m2logRj, 2))
    return (pv, m2logRj)

def calc_p_values(im_list):
    im_list = ee.List(im_list)
    k = im_list.length()
    def calc_ell(ell):
        ell = ee.Number(ell)
        im_list_ell = im_list.slice(k.subtract(ell), k)
        def calc_js(j):
            j = ee.Number(j)
            pv1, m2logRj1 = calc_p_value(im_list_ell, j)
            return ee.Feature(None, {'pv': pv1, 'm2logRj': m2logRj1})
        js = ee.List.sequence(2, ell)
        pv_m2logRj = ee.FeatureCollection(js.map(calc_js))
        m2logQl = ee.ImageCollection(pv_m2logRj.aggregate_array('m2logRj')).sum()
        pvQl = ee.Image.constant(1).subtract(qwe2cdf(m2logQl, ell.subtract(1).multiply(2)))
        pvs = ee.List(pv_m2logRj.aggregate_array('pv')).add(pvQl)
        return pvs
    ells = ee.List.sequence(k, 2, -1)
    pv_arr = ells.map(calc_ell)
    return pv_arr


def filter_x(current, prev):
    pv = ee.Image(current)
    prev = ee.Dictionary(prev)
    pvQ = ee.Image(prev.get('pvQ'))
    i = ee.Number(prev.get('i'))
    number_of_changes = ee.Image(prev.get('number_of_changes'))
    recent_changes = ee.Image(prev.get('recent_changes'))
    change_interval = ee.Image(prev.get('change_interval'))
    time_series_change = ee.Image(prev.get('time_series_change'))
    alpha = ee.Image(prev.get('alpha'))
    j = ee.Number(prev.get('j'))
    cmapj = number_of_changes.multiply(0).add(i.add(j).subtract(1))
    tst = pv.lt(alpha).And(pvQ.lt(alpha)).And(number_of_changes.eq(i.subtract(1)))
    number_of_changes = number_of_changes.where(tst, cmapj)
    change_interval = change_interval.where(tst, change_interval.add(1))
    recent_changes = ee.Algorithms.If(i.eq(1), recent_changes.where(tst, cmapj), recent_changes)   
    idx = i.add(j).subtract(2)
    tmp = time_series_change.select(idx)
    bname = time_series_change.bandNames().get(idx)
    tmp = tmp.where(tst, 1)
    tmp = tmp.rename([bname])
    time_series_change = time_series_change.addBands(tmp, [bname], True)
    return ee.Dictionary({'i': i, 'j': j.add(1), 'alpha': alpha, 'pvQ': pvQ,
                          'number_of_changes': number_of_changes, 'recent_changes': recent_changes, 'change_interval': change_interval, 'time_series_change':time_series_change})

def filter_y(current, prev):
    current = ee.List(current)
    pvs = current.slice(0, -1)
    pvQ = ee.Image(current.get(-1))
    prev = ee.Dictionary(prev)
    i = ee.Number(prev.get('i'))
    alpha = ee.Image(prev.get('alpha'))
    median = prev.get('median')
    pvQ = ee.Algorithms.If(median, pvQ.focal_median(2.5), pvQ)
    number_of_changes = prev.get('number_of_changes')
    recent_changes = prev.get('recent_changes')
    change_interval = prev.get('change_interval')
    time_series_change = prev.get('time_series_change')
    first = ee.Dictionary({'i': i, 'j': 1, 'alpha': alpha, 'pvQ': pvQ,
                           'number_of_changes': number_of_changes, 'recent_changes': recent_changes, 'change_interval': change_interval, 'time_series_change': time_series_change})
    result = ee.Dictionary(ee.List(pvs).iterate(filter_x, first))
    return ee.Dictionary({'i': i.add(1), 'alpha': alpha, 'median': median,
                          'number_of_changes': result.get('number_of_changes'), 'recent_changes': result.get('recent_changes'),
                          'change_interval': result.get('change_interval'), 'time_series_change': result.get('time_series_change')})

def dmap_process(current, prev):
    prev = ee.Dictionary(prev)
    j = ee.Number(prev.get('j'))
    image = ee.Image(current)
    avimg = ee.Image(prev.get('avimg'))
    diff = image.subtract(avimg)
    pos_diff = ee.Image(diff.select(0).gt(0).And(yz(diff).gt(0)))
    neg_diff = ee.Image(diff.select(0).lt(0).And(yz(diff).gt(0)))
    time_series_change = ee.Image(prev.get('time_series_change'))
    bmap_j = time_series_change.select(j)
    dmap = ee.Image.constant(ee.List.sequence(1, 3))
    bmap_j = bmap_j.where(bmap_j, dmap.select(2))
    bmap_j = bmap_j.where(bmap_j.And(pos_diff), dmap.select(0))
    bmap_j = bmap_j.where(bmap_j.And(neg_diff), dmap.select(1))
    time_series_change = time_series_change.addBands(bmap_j, overwrite=True)
    i = ee.Image(prev.get('i')).add(1)
    avimg = avimg.add(image.subtract(avimg).divide(i))
    avimg = avimg.where(bmap_j, image)
    i = i.where(bmap_j, 1)
    return ee.Dictionary({'avimg': avimg, 'time_series_change': time_series_change, 'j': j.add(1), 'i': i})

def generate_change_maps(im_list, median=False, alpha=0.0001):
    k = im_list.length()
    pv_arr = ee.List(calc_p_values(im_list))
    number_of_changes = ee.Image(im_list.get(0)).select(0).multiply(0)
    time_series_change = ee.Image.constant(ee.List.repeat(0, k.subtract(1))).add(number_of_changes)
    alpha = ee.Image.constant(alpha)
    first = ee.Dictionary({'i': 1, 'alpha': alpha, 'median': median,
                           'number_of_changes': number_of_changes, 'recent_changes': number_of_changes, 'change_interval': number_of_changes, 'time_series_change': time_series_change})
    result = ee.Dictionary(pv_arr.iterate(filter_y, first))
    time_series_change = ee.Image(result.get('time_series_change'))
    avimg = ee.Image(im_list.get(0))
    j = ee.Number(0)
    i = ee.Image.constant(1)
    first = ee.Dictionary({'avimg': avimg, 'time_series_change': time_series_change, 'j': j, 'i': i})
    dmap = ee.Dictionary(im_list.slice(1).iterate(dmap_process, first)).get('time_series_change')
    return ee.Dictionary(result.set('time_series_change', dmap))


# 대상지역 geojson, aoi

In [ ]:
geoJSON = {} # geojson 입력!!
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

## Sentinel-1 영상검색

In [ ]:
orbit_type = 'ASCENDING' # DESCENDING
relative_orbit = # relative orbit number 입력, 같은 RO에서 작성
start_date = datetime(2020, 1, 1)
days = 180
end_date = start_date + timedelta(days=days)
print('from', start_date, 'to', end_date)

image_collection = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
           .filterBounds(aoi)
           .filterDate(ee.Date(start_date), ee.Date(end_date))
           .filter(ee.Filter.eq('orbitProperties_pass', orbit_type))
           .filter(ee.Filter.eq('relativeOrbitNumber_start', relative_orbit))
           .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
           .sort('date'))

timestamp_list = (image_collection.aggregate_array('date')
                 .map(lambda d: ee.String('T').cat(ee.String(d)))
                 .getInfo())

image_list = image_collection.toList(image_collection.size())
image_list = ee.List(image_list.map(clip_image))

R_band = 0
B_band = image_list.length().getInfo() - 1
G_band = round(B_band / 2)
print(r'{0}, {1} = {2}'.format(orbit_type, 'k', image_list.length().getInfo()))
print(timestamp_list)

vv_list = image_list.map(select_aa)
location = aoi.centroid().coordinates().getInfo()[::-1]

map_ = folium.Map(location=location, zoom_start=11)

rgb_images = (ee.Image.rgb(vv_list.get(R_band), vv_list.get(B_band), vv_list.get(G_band))
              .log10().multiply(10))

map_.xz_abc_layer(rgb_images, {'min': -20, 'max': 0}, 'rgb composite')
map_.add_child(folium.LayerControl())


# 샘플지역 입력

In [ ]:
geoJSON_sample = {} # 샘플링 geojson을 입력, roi보다 작은 사이즈, 1e9보다 작아야 함.
coords_sample = geoJSON_sample['features'][0]['geometry']['coordinates']
aoi_sub = ee.Geometry.Polygon(coords_sample)
location = aoi_sub.centroid().coordinates().getInfo()[::-1]

def get_vv_samples(j):
    j = ee.Number(j)
    sj = vv_list.get(j.subtract(1))
    j_fact = j.pow(j).divide(j.subtract(1).pow(j.subtract(1)))
    sum_j = ee.ImageCollection(vv_list.slice(0, j)).reduce(ee.Reducer.sum())
    sum_jm1 = ee.ImageCollection(vv_list.slice(0, j.subtract(1))).reduce(ee.Reducer.sum())
    Rj = sum_jm1.pow(j.subtract(1)).multiply(sj).multiply(j_fact).divide(sum_j.pow(j)).pow(5)
    sample = (Rj.sample(region=aoi_sub, scale=10, numPixels=1e3, seed=123)
              .aggregate_array('VV_sum'))
    return sample


# 샘플링 평가

In [ ]:
k = round((image_list.length().getInfo()))
alpha = 0.05#1.0000450028524455e-05 # K=2 일때, 0.05, 0.001 사용.

def calc_omnibus(im_list, m=4.4):
    def log_image(current):
        return ee.Image(current).log()

    im_list = ee.List(im_list)
    k = im_list.length()
    klogk = k.multiply(k.log())
    klogk = ee.Image.constant(klogk)
    sum_logs = ee.ImageCollection(im_list.map(log_image)).reduce(ee.Reducer.sum())
    log_sum = ee.ImageCollection(im_list).reduce(ee.Reducer.sum()).log()
    return klogk.add(sum_logs).subtract(log_sum.multiply(k)).multiply(-2 * m)

al = 50
pl = 1000
pll = pl / al

hist = (calc_omnibus(vv_list.slice(0, k))
        .reduceRegion(ee.Reducer.fixedHistogram(0, al, pl), geometry=aoi_sub, scale=10)
        .get('constant')
        .getInfo())

a = np.array(hist)
x = a[:, 0]
y = a[:, 1] / np.sum(a[:, 1])

polyline = np.linspace(0, al, pl)
plt.figure(figsize=(6, 4), dpi=500)
plt.xlim((0, al))
plt.xlabel('$\chi^2$')
plt.ylabel('$Probability$ $f(x)$')
plt.plot(x, y, '*', color='black', label='{0}'.format(r'vv sampling'))
plt.plot(x, chi2.pdf(x, k - 1) / pll, 'r-', linewidth=1, label='$\chi^2$, $k = {0}$'.format(k))
plt.legend(loc='upper right')
plt.grid(color='0.95')
plt.show()

p_value = ee.Image.constant(1).subtract(qwe2cdf(calc_omnibus(vv_list), k - 1))
samples = ee.List.sequence(2, k).map(get_vv_samples)
np.set_printoptions(precision=4, suppress=True)
print(np.corrcoef(samples.getInfo()))


# 시계열 변화탐지

In [ ]:
result = generate_change_maps(image_list, median=True, alpha=alpha)
number_of_changes = ee.Image(result.get('number_of_changes'))
recent_changes = ee.Image(result.get('recent_changes'))
change_interval = ee.Image(result.get('change_interval'))
time_series_change = ee.Image(result.get('time_series_change'))
cmaps = ee.Image.cat(number_of_changes, recent_changes, change_interval, time_series_change).rename(['number_of_changes', 'recent_changes', 'change_interval'] + timestamp_list[1:])
location = aoi.centroid().coordinates().getInfo()[::-1]

change_interval_mask = change_interval.updateMask(change_interval.gt(0)) #변경 횟수
number_of_changes_mask = number_of_changes.updateMask(number_of_changes.gt(0)) #가장 최근에 변경된 간격
recent_changes_mask = recent_changes.updateMask(recent_changes.gt(0)) #첫 번째 변경 간격
time_series_change_mask = time_series_change.updateMask(time_series_change.gt(0)) #변경 (밴드단위)

result_map = folium.Map(location=location, zoom_start=11, tiles='cartodbpositron')

palette = ['darkblue', 'blue', 'cyan', 'yellow', 'orange', 'red']
colormap = cm.LinearColormap(colors=['darkblue', 'blue', 'cyan', 'yellow', 'orange', 'red', 'purple'],
                             index=np.linspace(1, image_list.length().getInfo(), 7), vmin=1, vmax=image_list.length().getInfo(),
                             caption='ACD, {} days'.format(k*12))
colormap.add_to(result_map)
result_map.xz_abc_layer(number_of_changes_mask, {'min': 0, 'max': image_list.length().getInfo() - 1, 'palette': palette}, 'number_of_changes')
result_map.xz_abc_layer(recent_changes_mask, {'min': 0, 'max': image_list.length().getInfo() - 1, 'palette': palette}, 'recent_changes')
result_map.xz_abc_layer(change_interval_mask, {'min': 0, 'max': (image_list.length().getInfo() - 1) / 2, 'palette': palette}, 'change_interval')
result_map.add_child(folium.LayerControl())


# 파일저장

In [ ]:
task = ee.batch.Export.image.toDrive(**{
    'image': time_series_change,
    'description': 'ACD_{0}_{1}'.format(orbit_type, 'time_series_change'),
    'folder':'Earthengine',
    'scale': 10,
    'region': aoi
})
task.start()